In [3]:
include("Create_Sample.jl")
include("MIP_SPLIT.jl")
include("Split.jl")
include("GA.jl")
include("Initial.jl")
include("Mutation.jl")
include("Crossover.jl")
include("Neighborhood.jl")
include("Neighborhood_intra.jl")
include("costs.jl")
# include("Draw.jl")
include("Escape.jl")
include("cross_escape.jl")
include("intersection.jl")
using XLSX

# function Write_to_excel(exfile, sheetnumber, instance_count, instance_name,m, best, avg, worst, run_time)
#     instance_count += 1
#     XLSX.openxlsx(exfile, mode="rw") do xf
#         sheet = xf[sheetnumber]
#         sheet["A"*string(instance_count)] = string(instance_name)
#         sheet["B"*string(instance_count)] = m
#         sheet["C"*string(instance_count)] = best
#         sheet["D"*string(instance_count)] = avg
#         sheet["E"*string(instance_count)] = worst
#         sheet["F"*string(instance_count)] = run_time
#     end
# end

In [2]:
function Create_random_sample(n::Int)
    allNodes = rand(n+1,2)
    num_of_nodes = n - 1
    T = Matrix{Float64}(undef, num_of_nodes + 2, num_of_nodes + 2)
    depot = allNodes[1, :]
    Nodes = allNodes[2:num_of_nodes+1, :]

    T[1, 1] = 0.0
    T[num_of_nodes+2, num_of_nodes+2] = 0.0
    T[1, num_of_nodes+2] = 0.0
    T[num_of_nodes+2, 1] = 0.0
    @inbounds for i in 1:num_of_nodes
        T[1, i+1] = euclidean(depot, Nodes[i, :])
        T[i+1, 1] = T[1, i+1]
        T[num_of_nodes+2, i+1] = T[1, i+1]
        T[i+1, num_of_nodes+2] = T[1, i+1]
        @inbounds for j in 1:num_of_nodes
            T[i+1, j+1] = euclidean(Nodes[i, :], Nodes[j, :])
            T[j+1, i+1] = T[i+1, j+1]
        end
    end

    return T, depot, Nodes
end

Create_random_sample (generic function with 1 method)

In [4]:
Ns = [200]
Mss = [[15, 20]]


best_v = Float64[]
Avg_v = Float64[]
worst_v = Float64[]

count = 0

P = Chromosome[]
best_chrm = Chromosome(Int[], 0.0, 0.0, Tour[])
worst_chrm = Chromosome(Int[], 0.0, 0.0, Tour[])
crossover_functions = [5,9]
demands = Int[]
W = 150
for (n_index, n) in enumerate(Ns)
    for (m_index, m) in enumerate(Mss[n_index])
        best_ = 0.0
        Avg_ = 0.0
        worst_ = 0.0
        time_ = 0.0
        println()
        println("Nc=$n, Nv=$m:")
        for instance = 1:100
            
            count += 1
            T, depot, Customers = Create_random_sample(n)
            n_nodes = n-1
            demands = ones(Int, n_nodes)
            W = 150
            h = 0.3
            popsize = (10,20)
            k_tournament = 2
            num_iter = 2500
            time_limit = Inf
            Mutation_Chance = 1.0

            num_runs = 10
            num_nei = 2
            avg = 0.0
            best = Inf
            worst = 0.0
            t1 = time() 
            P = Chromosome[]
            for i=1:num_runs
    #             println("run ", i)
                P, roullet = Perform_Genetic_Algorithm(T, demands,m, W, h, popsize, 
                    k_tournament, num_iter, time_limit, Mutation_Chance, num_nei, crossover_functions, Customers, depot);
                avg += P[1].fitness
                if P[1].fitness < best
                    best = P[1].fitness
                    best_chrm = P[1]
                end
                if P[1].fitness > worst
                    worst = P[1].fitness
                end
            end
            t2 = time()
            println("Instance $instance:  Best= ", round(best, digits = 3), "  Average= ", round(avg/num_runs, digits = 3), 
                "  Worst= ", round(worst, digits = 3), " , run time= ", round((t2-t1)/num_runs, digits=3))
#             Write_in_excel("random_results.xlsx", n_index, m_index, instance, best, avg/num_runs, worst, (t2-t1)/num_runs)
            best_ += best
            worst_ += worst
            Avg_ += (avg)/num_runs
            time_ += (t2-t1)/num_runs
        end
        println()
        println("Final results for Nc=$n, Nv=$m: best = $(best_/100),
            avg = $(Avg_/100), worst = $(worst_/100), time = $(time_/100)")
    end
end


Nc=200, Nv=15:
The best objective achieved in 3139 generations is: 2.138510273762085 and it took 28.17199993133545 seconds.
The best objective achieved in 2527 generations is: 2.138510273762085 and it took 21.407000064849854 seconds.
The best objective achieved in 2574 generations is: 2.138510273762085 and it took 22.009000062942505 seconds.
The best objective achieved in 2537 generations is: 2.138510273762085 and it took 21.098000049591064 seconds.
The best objective achieved in 2574 generations is: 2.138510273762085 and it took 21.874000072479248 seconds.
The best objective achieved in 2673 generations is: 2.138510273762085 and it took 22.384000062942505 seconds.
The best objective achieved in 2522 generations is: 2.138510273762085 and it took 21.45799994468689 seconds.
The best objective achieved in 2550 generations is: 2.1385102737620842 and it took 22.15499997138977 seconds.
The best objective achieved in 2502 generations is: 2.1385102737620842 and it took 21.122999906539917 seco

The best objective achieved in 2500 generations is: 2.3222138083136623 and it took 22.02900004386902 seconds.
The best objective achieved in 2500 generations is: 2.3222138083136623 and it took 21.31599998474121 seconds.
The best objective achieved in 2500 generations is: 2.3222138083136623 and it took 22.30900001525879 seconds.
The best objective achieved in 2500 generations is: 2.3222138083136628 and it took 21.71499991416931 seconds.
The best objective achieved in 2500 generations is: 2.322213808313662 and it took 22.23799991607666 seconds.
The best objective achieved in 2500 generations is: 2.3222138083136628 and it took 22.046000003814697 seconds.
The best objective achieved in 2500 generations is: 2.3222138083136623 and it took 21.878999948501587 seconds.
The best objective achieved in 2500 generations is: 2.3222138083136628 and it took 22.346999883651733 seconds.
The best objective achieved in 2500 generations is: 2.3222138083136628 and it took 22.509000062942505 seconds.
The bes

The best objective achieved in 2500 generations is: 2.3375042774128794 and it took 26.23300004005432 seconds.
The best objective achieved in 2500 generations is: 2.337504277412879 and it took 26.628000020980835 seconds.
The best objective achieved in 2500 generations is: 2.3375042774128794 and it took 27.003000020980835 seconds.
The best objective achieved in 2500 generations is: 2.3375042774128794 and it took 26.52300000190735 seconds.
The best objective achieved in 2500 generations is: 2.3375042774128794 and it took 26.251999855041504 seconds.
The best objective achieved in 2500 generations is: 2.3375042774128785 and it took 26.138000011444092 seconds.
The best objective achieved in 2500 generations is: 2.3375042774128794 and it took 26.657999992370605 seconds.
The best objective achieved in 2500 generations is: 2.3375042774128794 and it took 26.64199995994568 seconds.
The best objective achieved in 2500 generations is: 2.3375042774128794 and it took 26.448999881744385 seconds.
The b

The best objective achieved in 2500 generations is: 1.7392558704979144 and it took 22.049000024795532 seconds.
The best objective achieved in 2500 generations is: 1.7392558704979144 and it took 22.598999977111816 seconds.
The best objective achieved in 2500 generations is: 1.7392558704979146 and it took 21.829999923706055 seconds.
The best objective achieved in 2500 generations is: 1.7392558704979142 and it took 21.015000104904175 seconds.
The best objective achieved in 2500 generations is: 1.7392558704979144 and it took 22.384999990463257 seconds.
The best objective achieved in 2500 generations is: 1.7392558704979144 and it took 22.139999866485596 seconds.
The best objective achieved in 2500 generations is: 1.7392558704979144 and it took 22.064000129699707 seconds.
The best objective achieved in 2500 generations is: 1.7392558704979142 and it took 22.01699995994568 seconds.
The best objective achieved in 2500 generations is: 1.7392558704979142 and it took 21.594000101089478 seconds.
Th

The best objective achieved in 2500 generations is: 2.0337785524321452 and it took 21.969000101089478 seconds.
The best objective achieved in 2500 generations is: 2.0337785524321452 and it took 22.44599986076355 seconds.
The best objective achieved in 2500 generations is: 2.0337785524321452 and it took 22.23900008201599 seconds.
The best objective achieved in 2500 generations is: 2.0337785524321452 and it took 21.65400004386902 seconds.
The best objective achieved in 2500 generations is: 2.0337785524321452 and it took 22.342000007629395 seconds.
The best objective achieved in 2500 generations is: 2.0337785524321452 and it took 23.02300000190735 seconds.
The best objective achieved in 2500 generations is: 2.0337785524321452 and it took 22.682000160217285 seconds.
The best objective achieved in 2500 generations is: 2.0337785524321452 and it took 23.467999935150146 seconds.
The best objective achieved in 2500 generations is: 2.0337785524321452 and it took 23.134000062942505 seconds.
The b

...

The best objective achieved in 2500 generations is: 2.12744701741818 and it took 26.210999965667725 seconds.
The best objective achieved in 2500 generations is: 2.12744701741818 and it took 26.328999996185303 seconds.
The best objective achieved in 2500 generations is: 2.12744701741818 and it took 25.927000045776367 seconds.
The best objective achieved in 2500 generations is: 2.12744701741818 and it took 25.342000007629395 seconds.
The best objective achieved in 2500 generations is: 2.12744701741818 and it took 26.217999935150146 seconds.
The best objective achieved in 2500 generations is: 2.12744701741818 and it took 25.97000002861023 seconds.
The best objective achieved in 2500 generations is: 2.12744701741818 and it took 26.10800004005432 seconds.
The best objective achieved in 2500 generations is: 2.12744701741818 and it took 26.27500009536743 seconds.
The best objective achieved in 2500 generations is: 2.127447017418179 and it took 26.379000186920166 seconds.
The best objective ac

The best objective achieved in 2502 generations is: 2.1260670063635425 and it took 25.634000062942505 seconds.
The best objective achieved in 2575 generations is: 2.126067006363543 and it took 25.414999961853027 seconds.
The best objective achieved in 2501 generations is: 2.1260670063635425 and it took 24.426000118255615 seconds.
The best objective achieved in 2521 generations is: 2.126067006363543 and it took 25.23199987411499 seconds.
The best objective achieved in 2501 generations is: 2.126067006363543 and it took 25.917999982833862 seconds.
The best objective achieved in 2562 generations is: 2.1260670063635425 and it took 25.240000009536743 seconds.
The best objective achieved in 2505 generations is: 2.126067006363543 and it took 25.29199981689453 seconds.
The best objective achieved in 2504 generations is: 2.1260670063635434 and it took 24.51699995994568 seconds.
The best objective achieved in 2503 generations is: 2.126067006363543 and it took 25.072999954223633 seconds.
The best 

The best objective achieved in 2500 generations is: 2.0771750792595185 and it took 25.051000118255615 seconds.
The best objective achieved in 2500 generations is: 2.077175079259518 and it took 25.20300006866455 seconds.
The best objective achieved in 2500 generations is: 2.0771750792595176 and it took 25.616999864578247 seconds.
The best objective achieved in 2500 generations is: 2.077175079259518 and it took 24.835000038146973 seconds.
The best objective achieved in 2500 generations is: 2.077175079259518 and it took 25.431999921798706 seconds.
The best objective achieved in 2500 generations is: 2.077175079259518 and it took 25.072999954223633 seconds.
The best objective achieved in 2500 generations is: 2.077175079259518 and it took 24.33299994468689 seconds.
The best objective achieved in 2500 generations is: 2.077175079259518 and it took 24.817000150680542 seconds.
The best objective achieved in 2500 generations is: 2.077175079259518 and it took 24.54699993133545 seconds.
The best ob

The best objective achieved in 2500 generations is: 2.131792542665986 and it took 25.39199995994568 seconds.
The best objective achieved in 2500 generations is: 2.131792542665986 and it took 25.766000032424927 seconds.
The best objective achieved in 2500 generations is: 2.131792542665986 and it took 26.46399998664856 seconds.
The best objective achieved in 2500 generations is: 2.1317925426659854 and it took 26.10199999809265 seconds.
The best objective achieved in 2500 generations is: 2.131792542665986 and it took 26.453999996185303 seconds.
The best objective achieved in 2500 generations is: 2.131792542665986 and it took 25.963000059127808 seconds.
The best objective achieved in 2500 generations is: 2.1317925426659854 and it took 25.456000089645386 seconds.
The best objective achieved in 2500 generations is: 2.131792542665985 and it took 25.055999994277954 seconds.
The best objective achieved in 2500 generations is: 2.131792542665986 and it took 25.98200011253357 seconds.
The best obj

The best objective achieved in 2500 generations is: 1.6671993935147884 and it took 26.382000207901 seconds.
The best objective achieved in 2500 generations is: 1.6671993935147882 and it took 26.56500005722046 seconds.
The best objective achieved in 2500 generations is: 1.6671993935147884 and it took 26.31500005722046 seconds.
The best objective achieved in 2500 generations is: 1.6671993935147882 and it took 26.855000019073486 seconds.
The best objective achieved in 2500 generations is: 1.6671993935147884 and it took 26.358999967575073 seconds.
The best objective achieved in 2500 generations is: 1.6671993935147882 and it took 26.721999883651733 seconds.
The best objective achieved in 2500 generations is: 1.6671993935147884 and it took 27.070000171661377 seconds.
The best objective achieved in 2500 generations is: 1.6671993935147884 and it took 27.393999814987183 seconds.
The best objective achieved in 2500 generations is: 1.6671993935147882 and it took 26.750999927520752 seconds.
The be

In [56]:
2demands = ones(Int, 19)
W = 20
SPLIT(T, demands, 3, W, S)

(3.825499848100242, Tour[Tour([12, 10, 1, 2, 18, 3, 19], 3.1304691426509117), Tour([8, 5, 17, 16, 4, 15], 3.296465639961826), Tour([13, 7, 6, 9, 11, 14], 3.825499848100242)])

In [49]:
include("Split.jl")
SPLIT_test(T, 3, S)

(4.58182010892675, Tour[Tour([14, 9, 13, 19, 6, 12, 15], 4.523771799254393), Tour([5, 3, 8, 10, 17, 11, 4], 4.1875125906608215), Tour([2, 7, 1, 16, 18], 4.58182010892675)])

In [29]:
include("Split.jl")
demands = ones(Int, 10)
W = 20
SPLIT(T, demands, 3, W, tsp_tour)

(170.16000000000003, Tour[Tour([1], 128.88), Tour([9, 2, 4, 5, 6], 170.16000000000003), Tour([8, 7, 3, 10], 133.08)])

In [25]:
T[1,2] + T[2,10] + T[10, 1]

147.0

In [3]:
function test()
    mu = 20
    sigma = 20 
    k_tournament = 6
    num_nei = 2
    crossover_functions = [5,7]
    sheetnumber = 1
    hs = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
    for h in hs
        test_parameters(mu, sigma, h, k_tournament, num_nei, crossover_functions, sheetnumber)
        sheetnumber += 1
    end
end

test()

Results for eil51 ,m=2
Best: 222.7  Average: 226.0  Worst: 230.7 , run time= 4.0
Results for eil51 ,m=3
Best: 164.1  Average: 167.1  Worst: 170.2 , run time= 3.0
Results for eil51 ,m=5
Best: 120.8  Average: 123.0  Worst: 128.2 , run time= 2.0
Results for eil51 ,m=7
Best: 112.1  Average: 112.1  Worst: 112.1 , run time= 1.0
Results for berlin52 ,m=2
Best: 4179.0  Average: 4195.7  Worst: 4223.7 , run time= 3.0
Results for berlin52 ,m=3
Best: 3191.2  Average: 3191.2  Worst: 3191.2 , run time= 1.0
Results for berlin52 ,m=5
Best: 2440.9  Average: 2457.9  Worst: 2607.9 , run time= 1.0
Results for berlin52 ,m=7
Best: 2440.9  Average: 2440.9  Worst: 2440.9 , run time= 1.0
Results for eil76 ,m=2
Best: 282.2  Average: 285.8  Worst: 291.7 , run time= 5.0
Results for eil76 ,m=3
Best: 205.4  Average: 209.3  Worst: 217.3 , run time= 5.0
Results for eil76 ,m=5
Best: 146.6  Average: 152.9  Worst: 158.7 , run time= 3.0
Results for eil76 ,m=7
Best: 127.7  Average: 130.4  Worst: 135.9 , run time= 4.0
Resu

Results:  Best: 0.5  Average: 2.3  Worst: 4.5   run time: 67.8


In [2]:
function test_parameters(mu::Int, sigma::Int, h::Float64, k_tournament::Int, num_nei::Int, crossover_functions::Vector{Int}, sheetnumber::Int)
    instances = [:eil51, :berlin52, :eil76, :rat99]
    LKH3 = [[222.7, 159.6, 124.0, 112.1],[4110.2, 3184.2, 2440.9, 2440.9], [280.9, 196.7, 143.4, 128.2],[690.8, 523.3, 467.0,442.5]]
    Ms = [2, 3, 5, 7]

    best_v = Float64[]
    Avg_v = Float64[]
    worst_v = Float64[]

    count = 0
    best_ = 0.0
    Avg_ = 0.0
    worst_ = 0.0
    time_ = 0.0
    P = Chromosome[]
    # best_chrm = Chromosome(Int[], 0.0, 0.0, Tour[])
    # worst_chrm = Chromosome(Int[], 0.0, 0.0, Tour[])
    # roullet_ = zeros(Int, 5)
    # counter_n = zeros(Int, 25)
    # counter_cr = zeros(Int, 7)
    instance_count = 1
    for (i,instance) in enumerate(instances)
        for (j,K) in enumerate(Ms)
            
            count += 1
            T = Read_TSPLIB_instance(instance, 1)
            n = size(T)[1]-2
            demands = ones(Int, n)
            W = 150
            popsize = (mu, sigma)
            num_iter = 2000
            Mutation_Chance = 0.0
            num_runs = 10
            avg = 0.0
            best = Inf
            worst = 0.0
            t1 = time() 
            P = Chromosome[]
            for i=1:num_runs
                P, roullet = Perform_Genetic_Algorithm(T, demands,K, W, h, popsize, 
                    k_tournament, num_iter, Mutation_Chance, num_nei, crossover_functions);
    #             roullet_ = roullet_ + roullet
                avg += P[1].fitness
                if P[1].fitness < best
                    best = P[1].fitness
                    best_chrm = P[1]
                end
                if P[1].fitness > worst
                    worst = P[1].fitness
                    worst_chrm = P[1]
                end
            end
            t2 = time()
            println("Results for ", instance, " ,m=", K)
            println("Best: ", round(best, digits = 1), "  Average: ", round(avg/num_runs, digits = 1), 
                "  Worst: ", round(worst, digits = 1), " , run time= ", round((t2-t1)/num_runs, digits=0))
            best_ += 100*(best-LKH3[i][j])/LKH3[i][j]
            worst_ += 100*(worst-LKH3[i][j])/LKH3[i][j]
            Avg_ += 100*(avg/num_runs-LKH3[i][j])/LKH3[i][j]
            time_ += t2-t1
            Write_to_excel("h.xlsx", sheetnumber, instance_count, instance, K, best, avg/num_runs, worst, (t2-t1)/num_runs)
            instance_count += 1
        end
    end
    Write_to_excel("h.xlsx", sheetnumber, instance_count, "summary", "", best_/count, Avg_/count, worst_/count, time_/count)
    println("Results:  Best: ", round(best_/count, digits=1), "  Average: ", round(Avg_/count, digits=1), "  Worst: ", round(worst_/count, digits=1), "   run time: ", round(time_/count, digits=1))
end

test_parameters (generic function with 1 method)

In [163]:
Avg_v

1-element Vector{Float64}:
 4.221414418100858

In [157]:
include("Crossover.jl")
c = tour_crossover3(parent1, parent2, T, n)

4-element Vector{Tour}:
 Tour([15, 49, 20, 28, 19, 34, 35, 2, 27, 30, 25, 7, 21, 1, 37, 31], 164.032979029429)
 Tour([26, 47, 4, 36, 41, 39, 18, 17, 8, 33, 29, 9, 38, 32, 44, 14, 43, 16], 263.79734812967604)
 Tour([45, 11, 46, 3, 40, 12, 24, 13, 23, 42, 6, 22], 140.1044183646207)
 Tour([5, 10, 48, 50], 88.44695698759188)

In [158]:
nodes = Int[]
for tour in c
    println(find_tour_length(tour.Sequence, T), "   ", length(tour.Sequence))
    for i in tour.Sequence
        push!(nodes, i)
    end
end

println(length(nodes))

164.032979029429   16
263.79734812967604   18
140.10441836462073   12
88.44695698759186   4
50


In [120]:
outsiders

13-element Vector{Int64}:
  3
  6
 11
 12
 13
 22
 23
 24
 40
 42
 45
 46
 47

In [121]:
intersect(nodes, outsiders)

Int64[]

In [52]:
cities = [1]
index = 1
i = 1
seq = Int[]
seqs = Vector{Vector{Int}}()
while i <= cities[length(cities)]
    if i == cities[index]
        push!(seq, i)
        if i == cities[length(cities)]
            push!(seqs, seq)
        end
        i += 1
        index += 1
    else
        if length(seq)>0
            push!(seqs, seq)
            seq = Int[]
        end
        i += 1
    end
end
seqs

1-element Vector{Vector{Int64}}:
 [1]

In [50]:
tour = deepcopy(P[2].tours[1])

Tour([10, 48, 8, 33, 29, 9, 38, 32, 44, 14, 43, 16, 50], 141.15021864047716)

In [94]:
include("Crossover.jl")
t = deepcopy(P[2].tours[1])
Remove_cities_from_one_tour(t, [1,2,4,5,12,13], T, n)

117.6247269601199

In [96]:
t

Tour([10, 48, 8, 33, 29, 9, 38, 32, 44, 14, 43, 16, 50], 117.6247269601199)

In [95]:
t1 = [10, 48, 8, 33, 29, 9, 38, 32, 44, 14, 43, 16, 50]
t2 = [1,2,4,5,12,13]
deleteat!(t1, t2)
find_tour_length(t1, T)

117.62472696011989

In [80]:
seqs = [[1,2]]
tour = t.Sequence
cost = t.cost
for seq in seqs

    ns = length(seq)
    if seq[1] == 1
        if ns == 1
            println("A")
            println(cost)
            cost = cost - T[1, tour[1]+1] - T[tour[1]+1, tour[2]+1] + T[1, tour[2]+1]
            print(cost)
        else
            println("B")
            println(cost)
            cost = cost - T[1, tour[1]+1] - sum(T[tour[seq[i]]+1, tour[seq[i+1]]+1] for i=1:ns-1)- T[tour[seq[ns]]+1, tour[seq[ns]+1]+1] + T[1, tour[seq[ns]+1]+1]
            println(cost)
        end
    elseif seq[ns] == nt
        if ns == 1
            cost = cost - T[tour[nt]+1, n_nodes+2] - T[tour[nt-1]+1, tour[nt]+1] + T[tour[nt-1]+1, n_nodes+2]
        else
            cost = cost - T[tour[nt]+1, n_nodes+2] - sum(T[tour[seq[i]]+1, tour[seq[i+1]]+1] for i=1:ns-1) - T[tour[seq[1]-1]+1, tour[seq[1]]+1] + T[tour[seq[1]-1]+1, n_nodes+2] 
        end
    else
        if ns == 1
            cost = cost - T[tour[seq[1]-1]+1, tour[seq[1]]+1] - T[tour[seq[1]]+1, tour[seq[1]+1]+1] + T[tour[seq[1]-1]+1, tour[seq[1]+1]+1]
        else
            cost = cost - T[tour[seq[1]-1]+1, tour[seq[1]]+1] - sum(T[tour[seq[i]]+1, tour[seq[i+1]]+1] for i=1:ns-1) 
            - T[tour[seq[ns]]+1, tour[seq[ns]+1]+1] + T[tour[seq[1]-1]+1, tour[seq[ns]+1]+1]
        end
    end
end 

B
141.1501091684944
132.55355476801103


In [56]:
cost


141.14999969651163

In [62]:
t1 = [10, 48, 8, 33, 29, 9, 38, 32, 44, 14, 43, 16, 50]
find_tour_length(t1, T)

141.15021864047714

In [69]:
t1 = [8, 33, 29, 9, 38, 32, 44, 14, 43, 16, 50]
find_tour_length(t1, T)

132.5536642399938

In [79]:
seq = [1,2]
tour = [10, 48, 8, 33, 29, 9, 38, 32, 44, 14, 43, 16, 50]
println(T[1, tour[1]+1])
ns = length(seq)
println(sum(T[tour[seq[i]]+1, tour[seq[i+1]]+1] for i=1:ns-1))
println(T[tour[seq[ns]]+1, tour[seq[ns]+1]+1])
println(T[1, tour[seq[ns]+1]+1])
cost = 0
cost = cost - T[1, tour[1]+1] - sum(T[tour[seq[i]]+1, tour[seq[i+1]]+1] for i=1:ns-1)- T[tour[seq[ns]]+1, tour[seq[ns]+1]+1] + T[1, tour[seq[ns]+1]+1]

12.083045973594572
14.317821063276353
6.4031242374328485
24.20743687382041


-8.596554400483367

In [74]:
T[1,11]

12.083045973594572

In [75]:
T[11,49]

14.317821063276353

In [76]:
T[49,9]

6.4031242374328485

In [77]:
T[1,9]

24.20743687382041

In [130]:
sort(sample(1:10, 2, replace=false))

2-element Vector{Int64}:
 4
 6

In [5]:
function Calculate_new_cost_swap_two_updated(tour1::Vector{Int}, cost1::Float64, city11::Int, city12::Int, position1::Int, tour2::Vector{Int}, cost2::Float64, city21::Int, city22::Int, position2::Int,T::Matrix{Float64}, n_nodes::Int)
    nt1 = length(tour1)
    nt2 = length(tour2)
    new_cost11 = cost1
    new_cost12 = cost1
    new_cost21 = cost2
    new_cost22 = cost2
    if nt1 == 2
        new_cost11 = T[1, city21+1] + T[city21+1, city22+1] + T[city22+1, n_nodes+2] 
        new_cost12 = T[1, city22+1] + T[city22+1, city21+1] + T[city21+1, n_nodes+2] 
    else
        if position1 == 1
            new_cost11 += T[1, city21+1] + T[city21+1, city22+1]  + T[city22+1, tour1[3]+1] - T[1, city11+1] - T[city11+1, city12+1] - T[city12+1, tour1[3]+1]
            new_cost12 += T[1, city22+1] + T[city22+1, city21+1]  + T[city21+1, tour1[3]+1] - T[1, city11+1] - T[city11+1, city12+1] - T[city12+1, tour1[3]+1]
        elseif position1 == nt1-1
            new_cost11 += T[tour1[nt1-2]+1, city21+1] + T[city21+1, city22+1] + T[city22+1, n_nodes+2] - T[tour1[nt1-2]+1, city11+1] - T[city11+1, city12+1] - T[city12+1, n_nodes+2]
            new_cost12 += T[tour1[nt1-2]+1, city22+1] + T[city22+1, city21+1] + T[city21+1, n_nodes+2] - T[tour1[nt1-2]+1, city11+1] - T[city11+1, city12+1] - T[city12+1, n_nodes+2]
        else
            new_cost11 += T[tour1[position1-1]+1, city21+1] + T[city21+1, city22+1] + T[city22+1, tour1[position1+2]+1] - T[tour1[position1-1]+1, city11+1] - T[city11+1, city12+1] - T[city12+1, tour1[position1+2]+1]
            new_cost12 += T[tour1[position1-1]+1, city22+1] + T[city22+1, city21+1] + T[city21+1, tour1[position1+2]+1] - T[tour1[position1-1]+1, city11+1] - T[city11+1, city12+1] - T[city12+1, tour1[position1+2]+1]
        end
    end

    if nt2 == 2
        new_cost21 = T[1, city11+1] + T[city11+1, city12+1] + T[city12+1, n_nodes+2] 
        new_cost22 = T[1, city12+1] + T[city12+1, city11+1] + T[city11+1, n_nodes+2] 
    else
        if position2 == 1
            new_cost21 += T[1, city11+1] + T[city11+1, city12+1]  + T[city12+1, tour2[3]+1] - T[1, city21+1] - T[city21+1, city22+1] - T[city22+1, tour2[3]+1]
            new_cost22 += T[1, city12+1] + T[city12+1, city11+1]  + T[city11+1, tour2[3]+1] - T[1, city21+1] - T[city21+1, city22+1] - T[city22+1, tour2[3]+1]
        elseif position2 == nt2-1
            new_cost21 += T[tour2[nt2-2]+1, city11+1] + T[city11+1, city12+1] + T[city12+1, n_nodes+2] - T[tour2[nt2-2]+1, city21+1] - T[city21+1, city22+1] - T[city22+1, n_nodes+2]
            new_cost22 += T[tour2[nt2-2]+1, city12+1] + T[city12+1, city11+1] + T[city11+1, n_nodes+2] - T[tour2[nt2-2]+1, city21+1] - T[city21+1, city22+1] - T[city22+1, n_nodes+2]
        else
            new_cost21 += T[tour2[position2-1]+1, city11+1] + T[city11+1, city12+1] + T[city12+1, tour2[position2+2]+1] - T[tour2[position2-1]+1, city21+1] - T[city21+1, city22+1] - T[city22+1, tour2[position2+2]+1]
            new_cost22 += T[tour2[position2-1]+1, city12+1] + T[city12+1, city11+1] + T[city11+1, tour2[position2+2]+1] - T[tour2[position2-1]+1, city21+1] - T[city21+1, city22+1] - T[city22+1, tour2[position2+2]+1]
        end
    end
    if new_cost11 < new_cost12
        if new_cost21 < new_cost22
            return new_cost11, new_cost21, true, true
        else
            return new_cost11, new_cost22, true, false
        end
    else
        if new_cost21 < new_cost22
            return new_cost12, new_cost21, false, true
        else
            return new_cost12, new_cost22, false, false
        end
    end
end

function Calculate_new_cost_swap_three_updated(tour1::Vector{Int}, cost1::Float64, city11::Int, city12::Int, city13::Int, position1::Int, tour2::Vector{Int}, cost2::Float64, city21::Int, city22::Int, city23::Int, position2::Int,T::Matrix{Float64}, n_nodes::Int)
    nt1 = length(tour1)
    nt2 = length(tour2)
    new_cost11 = cost1
    new_cost12 = cost1
    new_cost21 = cost2
    new_cost22 = cost2
    if nt1 == 3
        new_cost11 = T[1, city21+1] + T[city21+1, city22+1] + T[city22+1, city23+1] + T[city23+1, n_nodes+2] 
        new_cost12 = T[1, city23+1] + T[city23+1, city22+1] + T[city22+1, city21+1] + T[city21+1, n_nodes+2]
    else
        if position1 == 1
            new_cost11 += T[1, city21+1] + T[city21+1, city22+1] + T[city22+1, city23+1]  + T[city23+1, tour1[4]+1] - T[1, city11+1] - T[city11+1, city12+1] - T[city12+1, city13+1] - T[city13+1, tour1[4]+1]
            new_cost12 += T[1, city23+1] + T[city23+1, city22+1] + T[city22+1, city21+1]  + T[city21+1, tour1[4]+1] - T[1, city11+1] - T[city11+1, city12+1] - T[city12+1, city13+1] - T[city13+1, tour1[4]+1]
        elseif position1 == nt1-2
            new_cost11 += T[tour1[nt1-3]+1, city21+1] + T[city21+1, city22+1] + T[city22+1, city23+1] + T[city23+1, n_nodes+2] - T[tour1[nt1-3]+1, city11+1] - T[city11+1, city12+1] - T[city12+1, city13+1] - T[city13+1, n_nodes+2]
            new_cost12 += T[tour1[nt1-3]+1, city23+1] + T[city23+1, city22+1] + T[city22+1, city21+1] + T[city21+1, n_nodes+2] - T[tour1[nt1-3]+1, city11+1] - T[city11+1, city12+1] - T[city12+1, city13+1] - T[city13+1, n_nodes+2]
        else
            new_cost11 += T[tour1[position1-1]+1, city21+1] + T[city21+1, city22+1]  + T[city22+1, city23+1] + T[city23+1, tour1[position1+3]+1] - T[tour1[position1-1]+1, city11+1] - T[city11+1, city12+1] - T[city12+1, city13+1] - T[city13+1, tour1[position1+3]+1]
            new_cost12 += T[tour1[position1-1]+1, city23+1] + T[city23+1, city22+1]  + T[city22+1, city21+1] + T[city21+1, tour1[position1+3]+1] - T[tour1[position1-1]+1, city11+1] - T[city11+1, city12+1] - T[city12+1, city13+1] - T[city13+1, tour1[position1+3]+1]
        end
    end

    if nt2 == 3
        new_cost21 = T[1, city11+1] + T[city11+1, city12+1] + T[city12+1, city13+1] + T[city13+1, n_nodes+2] 
        new_cost22 = T[1, city13+1] + T[city13+1, city12+1] + T[city12+1, city11+1] + T[city11+1, n_nodes+2] 
    else
        if position2 == 1
            new_cost21 += T[1, city11+1] + T[city11+1, city12+1] + T[city12+1, city13+1]  + T[city13+1, tour2[4]+1] - T[1, city21+1] - T[city21+1, city22+1] - T[city22+1, city23+1] - T[city23+1, tour2[4]+1]
            new_cost22 += T[1, city13+1] + T[city13+1, city12+1] + T[city12+1, city11+1]  + T[city11+1, tour2[4]+1] - T[1, city21+1] - T[city21+1, city22+1] - T[city22+1, city23+1] - T[city23+1, tour2[4]+1]
        elseif position2 == nt2-2
            new_cost21 += T[tour2[nt2-3]+1, city11+1] + T[city11+1, city12+1] + T[city12+1, city13+1] + T[city13+1, n_nodes+2] - T[tour2[nt2-3]+1, city21+1] - T[city21+1, city22+1] - T[city22+1, city23+1] - T[city23+1, n_nodes+2]
            new_cost22 += T[tour2[nt2-3]+1, city13+1] + T[city13+1, city12+1] + T[city12+1, city11+1] + T[city11+1, n_nodes+2] - T[tour2[nt2-3]+1, city21+1] - T[city21+1, city22+1] - T[city22+1, city23+1] - T[city23+1, n_nodes+2]
        else
            new_cost21 += T[tour2[position2-1]+1, city11+1] + T[city11+1, city12+1]  + T[city12+1, city13+1] + T[city13+1, tour2[position2+3]+1] - T[tour2[position2-1]+1, city21+1] - T[city21+1, city22+1] - T[city22+1, city23+1] - T[city23+1, tour2[position2+3]+1]
            new_cost22 += T[tour2[position2-1]+1, city13+1] + T[city13+1, city12+1]  + T[city12+1, city11+1] + T[city11+1, tour2[position2+3]+1] - T[tour2[position2-1]+1, city21+1] - T[city21+1, city22+1] - T[city22+1, city23+1] - T[city23+1, tour2[position2+3]+1]
        end
    end
    if new_cost11 < new_cost12
        if new_cost21 < new_cost22
            return new_cost11, new_cost21, true, true
        else
            return new_cost11, new_cost22, true, false
        end
    else
        if new_cost21 < new_cost22
            return new_cost12, new_cost21, false, true
        else
            return new_cost12, new_cost22, false, false
        end
    end
end


10-element Vector{Int64}:
 2
 2
 2
 1
 1
 1
 3
 2
 1
 3